In [29]:
import numpy as np
import random
import pandas as pd

import matplotlib.pyplot as plt

In [30]:
def process_Loss_2(beta,Y,Z,lambda_):
    m,n,xx=Z.shape
    sum_loss=0
    for i in range(m):
        loss=beta.T@Z[i]@beta-Y[i]
        sum_loss=sum_loss+loss*loss
    
    
    sum_loss=sum_loss+lambda_*process_beta1_2(beta,n,lambda_)
    print(sum_loss)
    
    return sum_loss

In [31]:
def process_beta2(beta):
    
    loss=beta.T@beta
    loss=math.sqrt(loss)
    
    return loss

In [32]:
def get_betaloss(w_gu,beta,P):
    
    loss=0
    sum_loss=0
    for i in range(P):
        loss=(w_gu[i]-beta[i])
        sum_loss=sum_loss+loss*loss
    
    sum_loss=sum_loss
    print(sum_loss)
    return sum_loss

In [33]:
def process_beta1_2(beta,p,lambda_):
    sum_loss=0
    for i in range(p):
        loss=math.sqrt(abs(beta[i]))
        sum_loss=sum_loss+loss
    return sum_loss

In [34]:
def process_Loss(beta,X,Y,Z,lambda_):
    m,n=X.shape
    sum_loss=0
    for i in range(m):
        loss=beta.T@Z[i]@beta+X[i]@beta-Y[i]
        sum_loss=sum_loss+loss*loss
    
    
    sum_loss=sum_loss+lambda_*process_beta1_2(beta,n,lambda_)
    print(sum_loss)
    
    return sum_loss

In [35]:
def process_Z(n,p):
    
    matrix_dim = (n, p, p)
    Z=np.empty(matrix_dim)
    
    for i in range(n):
        z=np.random.randn(p,1)
        Z_i=z@z.T
        Z[i]=Z_i
    
    return Z

In [36]:
def process_beta(sparse,p):
    
    o2=np.zeros(p)
    index=np.random.choice(p,size=sparse,replace=False,p=None)
    
    random_number=[]
    for i in range (sparse):
        random_number.append(random.choice([1, -1]))
    random_number=np.array(random_number) 
    
    o2[index]=random_number
    
    return o2,index

In [37]:
def process_X(n,p):
    
    X=np.random.randn(n,p)
    
    
    return X

In [38]:
def process_Y(n,p,Z,X,beta):
    
    Random=np.random.normal(0,1,n)
    
    A_beta=(Z@beta+X)
    matrix_dim = (n, 1)
    Y=np.empty(matrix_dim)
    
    for i in range(n):
        y_i=beta.T@Z[i]@beta+X[i]@beta+Random[i]
        Y[i]=y_i
    
    return Y

In [39]:
def Generate_omega_k(beta,K,h):
    p=len(beta)
    w_k=np.empty((K,p))
    for k in range(K):
        random_number=[]
        for i in range (p):
            random_number.append(random.choice([1, -1]))
        random_number=np.array(random_number)
        w_k[k]=beta+(h/p)*1.0*random_number 
    return w_k

In [40]:
def Generate_omega_k_trans(beta,K,h,beta_indice):
    p=len(beta)
    s=len(beta_indice)
    m=np.arange(0, p, 1)
    
    
    
    w_k=np.empty((K,p))
    for k in range(K):
        index=np.random.choice(np.delete(m,beta_indice,axis=0),size=10,replace=False,p=None)
        random_number=[]
        for i in range (p):
            random_number.append(random.choice([1, -1]))
        random_number=np.array(random_number)
        w_k[k]=(2*h/p)*1.0*random_number
        print(beta_indice)
        for j in beta_indice:
            w_k[k][j]=beta[j]+(2*h/p)*1.0*random_number[j]
        print(index)
        for l in index:
            w_k[k][l]=random.choice([1, -1])+(2*h/p)*1.0*random_number[l]
    return w_k

In [41]:
def pool_set(tr_k,ori_r):
    pool_tr=tr_k
    print(pool_tr.shape)
    l_tr=ori_r
    print(l_tr.shape)
    s_t=np.append(pool_tr,l_tr,axis=0)
    print(s_t.shape)
    return s_t
    

In [42]:
def Generate_k_XZY(n,p,w):
    X=process_X(n,p)
    Z=process_Z(n,p)
    Y=process_Y(n,p,Z,X,w)
    return X,Z,Y

In [43]:
def Generate_ori_XZY(n,p,beta):
    X=process_X(n,p)
    Z=process_Z(n,p)
    Y=process_Y(n,p,Z,X,w)

In [44]:
def train_split(X,Y,Z,n):
    
    x,p=X.shape
    num_list = list(range(n*3))
    random.shuffle(num_list)
    loc=np.array(num_list)
    iloc=np.array_split(loc,3)
    tr_x=np.empty((3,n,p))
    tr_y=np.empty((3,n,1))
    tr_z=np.empty((3,n,p,p))
    for i in range(3):
        tr_x[i]=X[iloc[i]]
        tr_y[i]=Y[iloc[i]]
        tr_z[i]=Z[iloc[i]]

    return tr_x,tr_y,tr_z

In [45]:
def generate_source_k(beta,n,p,k):
    num=n
    sr_X=np.empty((k,num,p))
    sr_Z=np.empty((k,num,p,p))
    sr_Y=np.empty((k,num,1))
    for i in range(k):
        sr_X[i],sr_Z[i],sr_Y[i]=Generate_k_XZY(num,p,beta[i])
    return sr_X,sr_Z,sr_Y

In [50]:
from sklearn.datasets import fetch_california_housing
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import math

def process_features(X):
    m, n = X.shape
    X = np.c_[np.ones((m, 1)), X]
    return X

class l12_IHTA:  
    def __init__(self, Lambda, maxIter=1000):
        self.lambda_ = Lambda
        self.maxIter = maxIter 
        
    def getphi(self,x,u):
        tem=pow(abs(x)/3.0,(-3.0/2))
        o=self.lambda_*u/8.0*tem
        return math.acos(o)
        
        
    def Getf(self,x,alpha):
        return 2/3*x*(1+math.cos(2/3*math.pi-2/3*self.getphi(x,alpha)))
    
        
    def soft_threshold(self,t, B_u,n):
            
        for i in range(n):
            
            s_1=(self.lambda_*t)
            s=pow(54,1/3)/4*pow(s_1,2/3)
            if abs(B_u[i])>s:
                B_u[i]=self.Getf(B_u[i],t)
    
            else :
                B_u[i]=0
                
        
        return B_u
        
    def fit(self, X, y,Z,alpha_):
        
        
        alpha=alpha_
        m,n = X.shape
        w = np.zeros(n)
        z = np.zeros(n)
        a=1
        for t in range(self.maxIter):
            sum_=0
            for i in range(m):
                s_1=z.T@Z[i]@z+X[i].T@z-y[i]
                s_2=Z[i]@z*2
                s_3=s_2+X[i]
                s_=s_1*s_3
                sum_=sum_+s_*2
            g=sum_/m
            
            r=z-alpha*g
            sw = w  
            
            
            w=self.soft_threshold(alpha, r,n)
            p = a
            
            a = (1+math.sqrt(1+4*p*p))/2
            z = w+(p-1)/a*(w-sw)
            
            w_loss=process_Loss(w,X,y,Z,self.lambda_)
            
        self.w=w
        
    def predict(self,X):
        return X@(self.w)
    
    def get(self):
        return self.w
    

In [51]:
class l12_IHTAbiasgai:  
    def __init__(self, Lambda, maxIter=1000):
        self.lambda_ = Lambda
        self.maxIter = maxIter 
        
    def getphi(self,x,u):
        tem=pow(abs(x)/3.0,(-3.0/2))
        o=self.lambda_*u/8.0*tem
        return math.acos(o)
        
        
    def Getf(self,x,alpha):
        return 2/3*x*(1+math.cos(2/3*math.pi-2/3*self.getphi(x,alpha)))
    
        
    def soft_threshold(self,t, B_u,n):
            
        for i in range(n):
            
            s_1=(self.lambda_*t)
            s=pow(54,1/3)/4*pow(s_1,2/3)
            if abs(B_u[i])>s:
                B_u[i]=self.Getf(B_u[i],t)
    
            else :
                B_u[i]=0
                
        
        return B_u
        
    def fit(self, X, y,Z,w_true,alpha_):
        
        alpha=alpha_
        m,n = X.shape
        w = w_true
        z = w_true
        a=1
        for t in range(self.maxIter):
            sum_=0
            for i in range(m):
                s_1=z.T@Z[i]@z+X[i].T@z-y[i]
                s_2=Z[i]@z*2
                s_3=s_2+X[i]
                s_=s_1*s_3
                sum_=sum_+s_*2
            g=sum_/m
            
            r=z-alpha*g
            sw = w  
            #z=w-alpha*g
            
            w=self.soft_threshold(alpha, r,n)
            p = a
            
            a = (1+math.sqrt(1+4*p*p))/2
            z = w+(p-1)/a*(w-sw)
            
            w_loss=process_Loss(w,X,y,Z,self.lambda_)
         
        self.w=w
        
    def predict(self,X):
        return X@(self.w)
    
    def get(self):
        return self.w

In [52]:
def bias_train(X,Y,Z,w_true,Lambda_,alpha_):
    
    #model = l12_IHTAbias(Lambda=0.001, maxIter=16)
    model = l12_IHTAbiasgai(Lambda=Lambda_, maxIter=300)
    model.fit(X, Y,Z,w_true,alpha_)
    sigma_true=model.get()
    #w_t=w_true+sigma_true
    w_t=sigma_true
    return w_t

In [53]:
def get_biasloss(w_t,beta):
    loss=0
    sum_loss=0
    for i in range(p):
        loss=(w_t[i]-beta[i])
        sum_loss=sum_loss+loss*loss
    
    sum_loss=sum_loss
    print(sum_loss)

In [54]:
n=100
p=600

sparse=10
beta,indice=process_beta(sparse,p)
X=process_X(n*3,p)
Z=process_Z(n*3,p)
Y=process_Y(n*3,p,Z,X,beta)
print(Y.shape)


(300, 1)


In [55]:
def pool_train(X,Y,Z,num):
    X_p=X
    Y_p=Y
    Z_p=Z
    for i in range (num):
        X_p=pool_set(Source_k_X[i],X_p)
        Y_p=pool_set(Source_k_Y[i],Y_p)
        Z_p=pool_set(Source_k_Z[i],Z_p)
    return X_p,Y_p,Z_p

In [56]:
h=10
K=1
w_k=Generate_omega_k(beta,K,h)
Source_k_X,Source_k_Z,Source_k_Y=generate_source_k(w_k,50,p,K)

In [57]:
model = l12_IHTA(Lambda=0.3, maxIter=300)
model.fit(X, Y,Z,1/2000)
w_ori1=model.get()

[79769.12127758]
[79763.41416387]
[79755.6135701]
[79745.55109752]
[79732.99531148]
[79717.64791127]
[79699.13549096]
[79676.99800918]
[79650.67400707]
[79619.48226136]
[79579.6985871]
[79530.04085281]
[79469.16025459]
[79395.31021076]
[79300.49813968]
[79171.98310745]
[78999.11894448]
[78758.67081377]
[78426.27657444]
[77944.52971089]
[77178.43047542]
[75830.9533416]
[73072.07576421]
[66589.77705743]
[53951.29967956]
[44731.70598804]
[38188.22390449]
[29894.43422826]
[22680.96820602]
[18585.65819863]
[16107.62555876]
[13274.93570006]
[10459.09206259]
[8658.90702294]
[7613.82264597]
[6683.86245877]
[5734.29979702]
[4851.90279793]
[4114.93561409]
[3560.49738962]
[3200.13524973]
[2968.28811951]
[2760.15870529]
[2540.40233166]
[2319.77376433]
[2105.53925512]
[1910.03232668]
[1737.2451271]
[1595.99660372]
[1485.63606542]
[1401.51641008]
[1328.51829036]
[1260.78231221]
[1190.87528669]
[1121.09426074]
[1049.49835743]
[981.13354393]
[923.45556174]
[878.81321957]
[846.04311667]
[822.22070493]


KeyboardInterrupt: 

In [ ]:
non_sum=np.count_nonzero(w_ori1)
print(non_sum)
indices = np.where(beta !=0)
print(w_ori1[indices])

In [ ]:
get_betaloss(w_ori1,beta,700)

In [ ]:
Train_X,Train_Y,Train_Z=train_split(X,Y,Z,100)
x_num,y_num=Train_X[0].shape
Ori_Train_X=np.empty((3,x_num*2,y_num))
Trx1=np.append(Train_X[1],Train_X[2],axis=0)
Ori_Train_X[0]=Trx1
Trx2=np.append(Train_X[0],Train_X[1],axis=0)
Ori_Train_X[1]=Trx2
Trx3=np.append(Train_X[1],Train_X[2],axis=0)
Ori_Train_X[2]=Trx3

Ori_Train_Z=np.empty((3,x_num*2,y_num,y_num))
Trz1=np.append(Train_Z[1],Train_Z[2],axis=0)
Ori_Train_Z[0]=Trz1
Trz2=np.append(Train_Z[0],Train_Z[2],axis=0)
Ori_Train_Z[1]=Trz2
Trz3=np.append(Train_Z[0],Train_Z[1],axis=0)
Ori_Train_Z[2]=Trz3

Ori_Train_Y=np.empty((3,x_num*2,1))
Try1=np.append(Train_Y[1],Train_Y[2],axis=0)
Ori_Train_Y[0]=Try1
Try2=np.append(Train_Y[0],Train_Y[2],axis=0)
Ori_Train_Y[1]=Try2
Try3=np.append(Train_Y[0],Train_Y[1],axis=0)
Ori_Train_Y[2]=Try3

In [ ]:
beta_indice=[]
for i in range(len(beta)):
    if beta[i] != 0:
        beta_indice.append(i)
beta_indice=np.array(beta_indice)

In [ ]:
h=10
K=1
w_k1=Generate_omega_k(beta,K,h)
Source_k_X,Source_k_Z,Source_k_Y=generate_source_k(w_k1,50,p,K)

In [ ]:
h=10
K_c=10-K
w_k2=Generate_omega_k_trans(beta,K_c,h,beta_indice)
Source_k_X_2,Source_k_Z_2,Source_k_Y_2=generate_source_k(w_k2,50,p,K_c)


In [ ]:
Source_k_X_pool=np.append(Source_k_X,Source_k_X_2,axis=0)
Source_k_Y_pool=np.append(Source_k_Y,Source_k_Y_2,axis=0)
Source_k_Z_pool=np.append(Source_k_Z,Source_k_Z_2,axis=0)

In [ ]:
print(Source_k_X_pool.shape)

In [ ]:

plt.plot(beta)

In [ ]:
def trans_target(Ori_Train_X,Ori_Train_Y,Ori_Train_Z):
    Loss_target=np.empty((3,1))
    for i in range (3):
        model = l12_IHTA(Lambda=0.15, maxIter=300)
        model.fit(Ori_Train_X[i],Ori_Train_Y[i],Ori_Train_Z[i],1/5000)
        w_true=model.get()
        Loss_target[i]=process_Loss(w_true,Train_X[i],Train_Y[i],Train_Z[i],0.15)
    return Loss_target,w_true

In [ ]:
Loss_target,w=trans_target(Ori_Train_X,Ori_Train_Y,Ori_Train_Z)

In [ ]:
plt.plot(w)
plt.plot(beta)

In [ ]:
def trans_source(Ori_Train_X,Ori_Train_Y,Ori_Train_Z):
    Loss_source=np.empty((3,10))
    w=np.empty((3,10,700))
    for i in range (3):
        for j in range(10):
            poolx_i_j=np.append(Ori_Train_X[i],Source_k_X_pool[j],axis=0)
            pooly_i_j=np.append(Ori_Train_Y[i],Source_k_Y_pool[j],axis=0)
            poolz_i_j=np.append(Ori_Train_Z[i],Source_k_Z_pool[j],axis=0)
            model = l12_IHTA(Lambda=0.15, maxIter=250)
            model.fit(poolx_i_j,pooly_i_j,poolz_i_j,1/5000)
            w_true=model.get()
            Loss_source[i][j]=process_Loss(w_true,Train_X[i],Train_Y[i],Train_Z[i],0.15)
            w[i][j]=w_true
    return Loss_source,w

In [ ]:
Loss_source,w_ll=trans_source(Ori_Train_X,Ori_Train_Y,Ori_Train_Z)

In [ ]:
Loss_K_source=np.zeros(10)
print(Loss_K_source)

In [ ]:

for i in range (3):
    for j in range(10):
        Loss_K_source[j]=Loss_K_source[j]+Loss_source[i][j]
print(Loss_K_source)        
Loss_K_source=Loss_K_source/3
print(Loss_K_source)

In [ ]:
Loss_0=np.sum(Loss_target)/3
print(Loss_0)


In [ ]:
A_indice=[]
for i in range(10):
    if(Loss_K_source[i]-Loss_0<=1000):
        A_indice.append(i)
print(A_indice)

In [ ]:
def get_trans_A(X,Y,Z,A_indice):
    X_dai=X
    Y_dai=Y
    Z_dai=Z
    for i in A_indice:
        X_dai=np.append(X_dai,Source_k_X_pool[i],axis=0)
        Y_dai=np.append(Y_dai,Source_k_Y_pool[i],axis=0)
        Z_dai=np.append(Z_dai,Source_k_Z_pool[i],axis=0)
    return X_dai,Y_dai,Z_dai

In [ ]:
Trans_X,Trans_Y,Trans_Z=get_trans_A(X,Y,Z,A_indice)


In [ ]:
print(Trans_X.shape)

In [ ]:
#strans
model = l12_IHTA(Lambda=0.15, maxIter=500)
model.fit(Trans_X, Trans_Y,Trans_Z,1/8000)
w_A_strans=model.get()

In [ ]:
plt.plot(w_A_strans)
plt.plot(beta)

In [ ]:
w_strans_A_2=bias_train(X,Y,Z,w_A_strans,1,1/8000)

In [ ]:
loss=0
sum_loss=0
for i in range(700):
    loss=(w_strans_A_2[i]-beta[i])
    sum_loss=sum_loss+loss*loss
    
sum_loss=sum_loss
print(sum_loss)

In [ ]:
plt.plot(w_A_strans)
plt.plot(beta)

In [ ]:
non_sum=np.count_nonzero(w_strans_A_2)
print(non_sum)
indices = np.where(beta !=0  )
print(w_strans_A_2[indices])

In [ ]:
def pool_train(X,Y,Z,num):
    X_p=X
    Y_p=Y
    Z_p=Z
    for i in range (num):
        X_p=pool_set(Source_k_X[i],X_p)
        Y_p=pool_set(Source_k_Y[i],Y_p)
        Z_p=pool_set(Source_k_Z[i],Z_p)
    return X_p,Y_p,Z_p

In [ ]:
X_p,Y_p,Z_p=pool_train(X,Y,Z,K)

In [ ]:
model = l12_IHTA(Lambda=0.07, maxIter=350)
model.fit(X_p, Y_p,Z_p,1/4000)
w_A_strans_h=model.get()

In [ ]:
get_betaloss(w_A_strans_h,beta,700)

In [ ]:
w_strans_A_2_h=bias_train(X,Y,Z,w_A_strans_h,3,1/6000)

In [ ]:
get_betaloss(w_strans_A_2_h,beta,700)

In [ ]:
non_sum=np.count_nonzero(w_strans_A_2_h)
print(non_sum)
indices = np.where(beta !=0)
print(w_strans_A_2_h[indices])